## **Instalação dos Pacotes**

In [ ]:
# !pip install -U -q langchain langchain-core langchain-openai langchain-community pydantic

## **Configuração das Variáveis de Ambiente**

Configuração da chave de API da OpenAI para autenticação

In [2]:
import os
from pathlib import Path


def load_env() -> None:
    """Load environment variables from the appropriate source.
    
    If running in Google Colab, loads secrets from Colab's userdata.
    Otherwise, loads variables from a local .env file using python-dotenv.
    """
    try:
        import google.colab
        from google.colab import userdata

        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_KEY")

    except ImportError:
        from dotenv import load_dotenv
        load_dotenv()


load_env()

## **Imports dos Pacotes Usados no Notebook**

Importação dos componentes principais do LangChain para construção de aplicações com LLMs

In [5]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

## **Inicialização do Modelo LLM**

Criação da instância do modelo ChatGPT que será utilizado em todos os exemplos

In [6]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4.1-nano")

print("Modelo LLM inicializado com sucesso!")

Modelo LLM inicializado com sucesso!


In [7]:
llm.invoke("olá, tudo bem?")

AIMessage(content='Olá! Tudo bem, e você? Como posso ajudar?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 13, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_336de151a4', 'id': 'chatcmpl-DCymJlKZTtGqjJXov2Gv06Ays14Zj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9297-aae8-75e2-9aa1-4a0e374b3599-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 13, 'output_tokens': 12, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
file_url = "https://raw.githubusercontent.com/renansantosmendes/synapseai_data/refs/heads/main/company_context.txt"

In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(file_url, encoding="utf-8")
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
from pydantic import BaseModel, Field
from typing import List, Dict

class ProductAnalysis(BaseModel):
    """Associação de um produto aos seus problemas."""
    product_name: str = Field(description="Nome do produto")
    problems: List[str] = Field(description="Lista de problemas identificados para este produto")

class ProductsProblems(BaseModel):
    """Estrutura final para a extração de múltiplos produtos."""
    content: List[ProductAnalysis] = Field(description="Lista de produtos e seus problemas")

In [ ]:
llm_structured = llm.with_structured_output(ProductsProblems)

In [ ]:
products_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes da empresa gere a descrição do
produto e seu nome. O nome deve ser o mais curto possível, sem adicionar a descrição.
O campo descrição deve ser feito de forma separada do nome do produto

A descrição deve conter todos os detalhes técnicos e não técnicos possíveis sobre o produto em questão.

Garanta que nenhuma informação seja criada, use somente o contexto da empresa para extrair as informações
</context>

<company_description>
{company_description}
</company_description>
""")

In [ ]:
class ProductEntry(BaseModel):
    """Representa um produto individual da companhia e sua definição oficial."""
    name: str = Field(
        description="O nome oficial do produto (ex: 'Cartão Platinum', 'App Invest', 'CDB Fácil')"
    )
    description: str = Field(
        description="A descrição detalhada das funcionalidades e regras do produto encontrada no texto"
    )

class CompanyProductCatalog(BaseModel):
    """Catálogo completo de produtos extraído dos documentos da empresa."""
    products: List[ProductEntry] = Field(description="Lista de todos os produtos identificados no texto")

In [ ]:
structured_llm = llm.with_structured_output(CompanyProductCatalog)
product_chain = products_prompt | structured_llm

## Criação da lista de produtos + descrições

In [ ]:
catalog = product_chain.invoke(
    {
        "company_description": docs[0].page_content
    }
)

In [ ]:
catalog.products

In [ ]:
catalog_dict = {item['name']: item['description'] for item in catalog.model_dump()["products"]}

In [ ]:
current_product = "FinBrain"
catalog_dict.get(current_product)

## Criação do público de cada produto

In [ ]:
class CustomerAnalysis(BaseModel):
    """Análise detalhada de um cliente potencial para um produto."""
    customer_name: str = Field(description="Nome do cliente")
    customer_description: str = Field(description="Descrição detalhada do cliente")

class PublicTargetAnalysis(BaseModel):
    """Análise detalhada do público-alvo de um produto, incluindo jornada do cliente e exemplos reais."""
    target_audience_description: str = Field(description="Descrição detalhada do público-alvo para o produto")
    customer_journey: str = Field(description="Descrição da jornada de uso do produto, desde o primeiro contato até o uso recorrente")
    example_customers: List[CustomerAnalysis] = Field(
        description="Lista de possíveis clientes reais que se encaixariam no público-alvo, com breve descrição de cada um"
    )

In [ ]:
customer_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes da empresa gere uma descrição
de um público alvo para o produto, uma possivel jornada de uso do produto, 
desde o primeiro contato do cliente com o produto, até o uso recorrente do mesmo.
Além disso, gere uma lista de possíveis clientes que sejam reais e que se encaixariam 
no público alvo do produto com uma breve descrição de cada um desses clientes.
</context>

<company_description>
{company_description}
</company_description>

Considere o produto "{product_name}" e sua descrição "{product_description}" para conseguir gerar problemas mais
contextualizados
""")

In [ ]:
customer_structured_llm = llm.with_structured_output(PublicTargetAnalysis)
customer_chain = customer_prompt | customer_structured_llm

In [ ]:
customers = customer_chain.invoke(
    {
        "company_description": docs[0].page_content,
        "product_name": current_product,
        "product_description": catalog_dict.get(current_product)
    }
)

In [ ]:
customers.model_dump()

# usar esse for aqui

In [ ]:
for current_product in catalog_dict.keys():
    print(current_product)

## Criação dos problemas para cada produto

In [ ]:
problems_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes de uma empresa gere uma lista de
possíveis problemas que cada produto da empresa pode apresentar do ponto de
vista do usuário final, como por exemplo API instável, sistema demora muito
para responder,
</context>

<company_description>
{company_description}
</company_description>

Considere o produto "{product_name}" e sua descrição "{product_description}" para conseguir gerar problemas mais
contextualizados. Garanta que os problemas listados sejam realmente relacionados ao produto e não sejam problemas genéricos de mercado.

Use as informações a seguir para conseguir gerar os problemas de cada produto:

## Informações sobre o produto:
- Descrição do público-alvo: {target_audience_description}
- Jornada do cliente: {customer_journey}
- Exemplos de clientes: {example_customers}
""")

In [ ]:
problems_structured_llm = llm.with_structured_output(ProductsProblems)
problems_chain = problems_prompt | problems_structured_llm

In [ ]:
problems_response = problems_chain.invoke({
    "product_name" : current_product,
    "product_description": catalog_dict.get(current_product),
    "company_description": docs[0].page_content,
    "target_audience_description": target_audience_dict.get(current_product),
    "customer_journey": customer_journey_dict.get(current_product),
    "example_customers": example_customers_dict.get(current_product),
})

In [ ]:
problems_response.model_dump()["content"]

In [ ]:
product_problems_dict = {item['product_name']: item['problems'] for item in problems_response.model_dump()["content"]}

In [ ]:
product_problems_dict

In [ ]:
import random

def get_random_element(input_list):
    """Retorna um elemento aleatório de uma lista."""
    if not input_list:
        return None
    return random.choice(input_list)

# Exemplo de uso com seus produtos:
meus_produtos = ["App Invest", "Cartão Platinum", "CDB Fácil", "Seguro Vida"]

# Toda vez que chamar, virá um diferente:
produto_sorteado = get_random_element(meus_produtos)
print(f"Produto escolhido: {produto_sorteado}")

In [ ]:
import pandas as pd
import json
from enum import Enum
from typing import List
from datetime import datetime
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. Definição das Classes com Regras de Negócio
class CriticalityLevel(str, Enum):
    LOW = "Baixa"
    MEDIUM = "Média"
    HIGH = "Alta"
    CRITICAL = "Crítica"

class SupportTicket(BaseModel):
    """Estrutura de um ticket de suporte com dados realistas."""
    customer_name: str = Field(description="Nome completo real (ex: João Carlos de Alencar, Maria Aparecida Souza)")
    customer_email: str = Field(description="E-mail variado (NÃO use @example.com. Use @gmail, @uol, @empresa.com.br, etc)")
    product_name: str = Field(description="Nome do produto do mercado financeiro ou tecnologia")
    issue_description: str = Field(description="Relato do problema em linguagem natural humana")
    opening_date: str = Field(description="Data de abertura (formato ISO YYYY-MM-DD)")
    criticality: CriticalityLevel = Field(description="Nível de urgência")

class TicketBatch(BaseModel):
    """Lote de tickets para exportação."""
    tickets: List[SupportTicket]

# 2. Configuração do Modelo e Prompt Estratégico
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9) # Temperatura alta para mais criatividade nos nomes
structured_llm = llm.with_structured_output(TicketBatch)

prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um gerador de dados sintéticos de alta fidelidade para o mercado brasileiro.

    DIRETRIZES PARA REALISMO:
    - NOMES: Use nomes brasileiros reais e diversos (sobrenomes como Silva, Oliveira, Cavalcanti, Schmidt, Nogueira).
    - E-MAILS: Use domínios como @hotmail.com, @uol.com.br, @terra.com.br, @gmail.com ou domínios de empresas fictícias (@startup.io, @banco.com.br).
    - PROIBIÇÃO: Está proibido o uso do domínio '@example.com'.
    - DATAS: Gere datas aleatórias no ano de 2025.
    - PERSONAS: Varie entre clientes corporativos sérios e usuários comuns informais."""),
    ("human", "Gere {number_of_tickets} tickets de suporte para o produto '{product_name}' com a seguinte descrição '{product_description}' sobre o problema: '{product_problem}'.")
])

# 3. Execução e Conversão
chain = prompt | structured_llm

# Exemplo de uso para seu projeto de NER
input_data = {
    "number_of_tickets": 5,
    "product_name": "FinBrain",
    "product_description":
    "product_problem": "Erro ao tentar liquidar ordem de compra de ações"
}

resultado = chain.invoke(input_data)

# 4. Transformação em Lista de Dicionários e DataFrame
# Usando model_dump para cada objeto da lista
tickets_dicts = [t.model_dump() for t in resultado.tickets]

df = pd.DataFrame(tickets_dicts)

# Exibindo o resultado para conferência
print(df[['customer_name', 'customer_email', 'criticality']])

# Opcional: Salvar para CSV para usar nos seus slides ou treino de modelo
# df.to_csv("tickets_sinteticos.csv", index=False)

In [ ]:
resultado.tickets[3].model_dump()